In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [2]:
import os
from dotenv import load_dotenv
import openai
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
ZILLIZ_URI = os.getenv("ZILLIZ_CLUSTER_01_URI")
ZILLIZ_TOKEN = os.getenv("ZILLIZ_CLUSTER_01_TOKEN")

In [3]:
from langchain.vectorstores import Milvus
embeddings = OpenAIEmbeddings()

In [7]:
vector_db = Milvus.from_documents(
    {},
    embeddings,
    connection_args={"uri": ZILLIZ_URI, 
                     "token": ZILLIZ_TOKEN}
)

In [8]:
retriever = Milvus.as_retriever(vectordb, search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [9]:
about_me = [
    {"input": "My favorite snack is chocolate",
     "output": "Nice"},
    {"input": "My favorite sport is swimming",
     "output": "Cool"},
    {"input": "My favorite beer is Guinness",
     "output": "Great"},
    {"input": "My favorite dessert is cheesecake",
     "output": "Good to know"},
    {"input": "My favorite musician is Taylor Swift",
     "output": "Same"}
]
for example in about_me:
    memory.save_context({"input": example["input"]}, {"output": example["output"]})

In [10]:
print(memory.load_memory_variables({"prompt": "who is my favorite musician?"})["history"])

input: My favorite musician is Taylor Swift
output: Same


In [11]:
llm = OpenAI(temperature=0) # Can be any valid LLM
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    memory=memory,
    verbose=True
)
conversation_with_summary.predict(input="Hi, my name is Gary, what's up?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite snack is chocolate
output: Nice

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi, my name is Gary, what's up?
AI:

> Finished chain.


" Hi Gary, I'm doing great. How about you?"

In [12]:
conversation_with_summary.predict(input="who is my favorite musician?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite musician is Taylor Swift
output: Same

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: who is my favorite musician?
AI:

> Finished chain.


' You said your favorite musician is Taylor Swift.'

In [13]:
conversation_with_summary.predict(input="Whats my favorite dessert?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite dessert is cheesecake
output: Good to know

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Whats my favorite dessert?
AI:

> Finished chain.


' You said your favorite dessert is cheesecake.'

In [14]:
conversation_with_summary.predict(input="What's my name?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: Hi, my name is Gary, what's up?
response:  Hi Gary, I'm doing great. How about you?

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: What's my name?
AI:

> Finished chain.


' Your name is Gary.'